# Yazılı & Sözlü Soru Önergeleri - Data Collection

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import numpy as np
import time

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(30)

### Dönemler:
- **22. Dönem Tüm Yasama Yılları** $>$ value = **220**
- **23. Dönem Tüm Yasama Yılları** $>$ value = **230**
- **24. Dönem Tüm Yasama Yılları** $>$ value = **240**
- **25. Dönem Tüm Yasama Yılları** $>$ value = **250**
- **26. Dönem Tüm Yasama Yılları** $>$ value = **260**

### Önerge Grubu:
- **Sözlü soru önergesi** $>$ value = **6** --> "sözlü"
- **Yazılı soru önergesi** $>$ value = **7** --> "yazılı"

In [3]:
def get_sonuc_page(donem, onerge_grubu):
    
    donem = str(donem) + "0"
    
    # Open "Geçmiş Dönem Yazılıve Sözlü Soru Önergeleri" page
    driver.get("https://www.tbmm.gov.tr/Denetim/Gecmis-Donem-Yazili-Soru-Onergeleri")
    
    # Select "Dönemi ve Yasama Yılı" from dropdown menu
    select_donem = Select(driver.find_element(By.XPATH, """//*[@id="DonemYasamaYili"]"""))
    select_donem.select_by_value(donem)
    
    
    # Select "Yazılı Soru Önergesi" or "Sözlü Soru Önergesi"
    if onerge_grubu == "yazılı":
        grup = str(7)
    elif onerge_grubu == "sözlü":
        grup = str(6)
    select_onerge_grubu = Select(driver.find_element(By.XPATH, """//*[@id="OnergeGrubu"]"""))
    select_onerge_grubu.select_by_value(grup)
    
    # Find "Sorguyu Başlat" button
    input_search = driver.find_element(By.XPATH, """//*[@id="main-area"]/form/div[11]/div/input[2]""")

    # Click "Sorguyu Başlat" button
    input_search.click()
    
    # Get total page size
    try:
        page_size = int(driver.find_element(By.XPATH, """//*[@id="dataTable_paginate"]/ul/li[8]/a""").text)
    except NoSuchElementException:
        # if there are two pages:
        page_size = int(driver.find_element(By.XPATH, """//*[@id="dataTable_paginate"]/ul/li[3]/a""").text)
    
    return page_size

In [4]:
def get_table_contents():
    # Find table
    table = driver.find_element(By.XPATH, """//*[@id="dataTable"]/tbody""")
    
    # Get rows of table
    rows = None
    while rows is None:
        try:
            rows = [row.text for row in table.find_elements(By.TAG_NAME, "tr")]
        except:
            pass
    
    # Get data from rows of the table
    df = pd.DataFrame()
    
    for i, row in enumerate(rows):
        #print(f"Row: {i+1}/{len(rows)}")

        col_1 = row.split("\n")[0].split(" ")
        col_2 = row.split("\n")
        
        donemi_ve_yasama_yili = col_1[0]
        esas_no = col_1[1]
        try:
            baskanliga_gelis_tarihi = col_1[2]
        except:
            baskanliga_gelis_tarihi = np.nan
        
        milletvekili = col_2[1]
        ozeti = col_2[2]
        
        df = df.append(pd.Series([donemi_ve_yasama_yili, esas_no, baskanliga_gelis_tarihi, milletvekili, ozeti],
                                 index=["Donemi_ve_Yasama_Yili", "Esas_No", "Baskanliga_Gelis_Tarihi", "Milletvekili", "Ozeti"]),
                       ignore_index=True)
        
    return df

In [5]:
def collect_data(donem, onerge_grubu):
    onergeler = pd.DataFrame()
    
    page_size = get_sonuc_page(donem=donem, onerge_grubu=onerge_grubu)
    
    for i in range(page_size):
        time.sleep(1)
        print(f"Page: {i+1}/{page_size} | {(i+1)/page_size*100:.2f}%")
        
        onergeler = onergeler.append(get_table_contents(), ignore_index=True).reset_index(drop=True)
        
        # Find "Sonraki" button
        next_button = driver.find_element(By.XPATH, """//*[@id="dataTable_next"]""")
    
        # Click "Sonraki" button
        next_button.click()
        
    onergeler["Onerge_Grubu"] = onerge_grubu
        
    print(f"Dönem: {donem} | Toplam Önerge Sayısı: {onergeler.shape[0]}\n\n")
        
    return onergeler

In [6]:
onerge_df = pd.DataFrame()

for donem in range(22, 27):
    for onerge_grubu in ["yazılı", "sözlü"]:
        print(f"Dönem: {donem} | Önerge Grubu: {onerge_grubu}\n----")
        onerge_df = onerge_df.append(collect_data(donem=donem, onerge_grubu=onerge_grubu), ignore_index=True).reset_index(drop=True)

Dönem: 22 | Önerge Grubu: yazılı
----
Page: 1/460 | 0.22%
Page: 2/460 | 0.43%
Page: 3/460 | 0.65%
Page: 4/460 | 0.87%
Page: 5/460 | 1.09%
Page: 6/460 | 1.30%
Page: 7/460 | 1.52%
Page: 8/460 | 1.74%
Page: 9/460 | 1.96%
Page: 10/460 | 2.17%
Page: 11/460 | 2.39%
Page: 12/460 | 2.61%
Page: 13/460 | 2.83%
Page: 14/460 | 3.04%
Page: 15/460 | 3.26%
Page: 16/460 | 3.48%
Page: 17/460 | 3.70%
Page: 18/460 | 3.91%
Page: 19/460 | 4.13%
Page: 20/460 | 4.35%
Page: 21/460 | 4.57%
Page: 22/460 | 4.78%
Page: 23/460 | 5.00%
Page: 24/460 | 5.22%
Page: 25/460 | 5.43%
Page: 26/460 | 5.65%
Page: 27/460 | 5.87%
Page: 28/460 | 6.09%
Page: 29/460 | 6.30%
Page: 30/460 | 6.52%
Page: 31/460 | 6.74%
Page: 32/460 | 6.96%
Page: 33/460 | 7.17%
Page: 34/460 | 7.39%
Page: 35/460 | 7.61%
Page: 36/460 | 7.83%
Page: 37/460 | 8.04%
Page: 38/460 | 8.26%
Page: 39/460 | 8.48%
Page: 40/460 | 8.70%
Page: 41/460 | 8.91%
Page: 42/460 | 9.13%
Page: 43/460 | 9.35%
Page: 44/460 | 9.57%
Page: 45/460 | 9.78%
Page: 46/460 | 10.00%
Page

Page: 363/460 | 78.91%
Page: 364/460 | 79.13%
Page: 365/460 | 79.35%
Page: 366/460 | 79.57%
Page: 367/460 | 79.78%
Page: 368/460 | 80.00%
Page: 369/460 | 80.22%
Page: 370/460 | 80.43%
Page: 371/460 | 80.65%
Page: 372/460 | 80.87%
Page: 373/460 | 81.09%
Page: 374/460 | 81.30%
Page: 375/460 | 81.52%
Page: 376/460 | 81.74%
Page: 377/460 | 81.96%
Page: 378/460 | 82.17%
Page: 379/460 | 82.39%
Page: 380/460 | 82.61%
Page: 381/460 | 82.83%
Page: 382/460 | 83.04%
Page: 383/460 | 83.26%
Page: 384/460 | 83.48%
Page: 385/460 | 83.70%
Page: 386/460 | 83.91%
Page: 387/460 | 84.13%
Page: 388/460 | 84.35%
Page: 389/460 | 84.57%
Page: 390/460 | 84.78%
Page: 391/460 | 85.00%
Page: 392/460 | 85.22%
Page: 393/460 | 85.43%
Page: 394/460 | 85.65%
Page: 395/460 | 85.87%
Page: 396/460 | 86.09%
Page: 397/460 | 86.30%
Page: 398/460 | 86.52%
Page: 399/460 | 86.74%
Page: 400/460 | 86.96%
Page: 401/460 | 87.17%
Page: 402/460 | 87.39%
Page: 403/460 | 87.61%
Page: 404/460 | 87.83%
Page: 405/460 | 88.04%
Page: 406/4

Page: 212/385 | 55.06%
Page: 213/385 | 55.32%
Page: 214/385 | 55.58%
Page: 215/385 | 55.84%
Page: 216/385 | 56.10%
Page: 217/385 | 56.36%
Page: 218/385 | 56.62%
Page: 219/385 | 56.88%
Page: 220/385 | 57.14%
Page: 221/385 | 57.40%
Page: 222/385 | 57.66%
Page: 223/385 | 57.92%
Page: 224/385 | 58.18%
Page: 225/385 | 58.44%
Page: 226/385 | 58.70%
Page: 227/385 | 58.96%
Page: 228/385 | 59.22%
Page: 229/385 | 59.48%
Page: 230/385 | 59.74%
Page: 231/385 | 60.00%
Page: 232/385 | 60.26%
Page: 233/385 | 60.52%
Page: 234/385 | 60.78%
Page: 235/385 | 61.04%
Page: 236/385 | 61.30%
Page: 237/385 | 61.56%
Page: 238/385 | 61.82%
Page: 239/385 | 62.08%
Page: 240/385 | 62.34%
Page: 241/385 | 62.60%
Page: 242/385 | 62.86%
Page: 243/385 | 63.12%
Page: 244/385 | 63.38%
Page: 245/385 | 63.64%
Page: 246/385 | 63.90%
Page: 247/385 | 64.16%
Page: 248/385 | 64.42%
Page: 249/385 | 64.68%
Page: 250/385 | 64.94%
Page: 251/385 | 65.19%
Page: 252/385 | 65.45%
Page: 253/385 | 65.71%
Page: 254/385 | 65.97%
Page: 255/3

Page: 138/1283 | 10.76%
Page: 139/1283 | 10.83%
Page: 140/1283 | 10.91%
Page: 141/1283 | 10.99%
Page: 142/1283 | 11.07%
Page: 143/1283 | 11.15%
Page: 144/1283 | 11.22%
Page: 145/1283 | 11.30%
Page: 146/1283 | 11.38%
Page: 147/1283 | 11.46%
Page: 148/1283 | 11.54%
Page: 149/1283 | 11.61%
Page: 150/1283 | 11.69%
Page: 151/1283 | 11.77%
Page: 152/1283 | 11.85%
Page: 153/1283 | 11.93%
Page: 154/1283 | 12.00%
Page: 155/1283 | 12.08%
Page: 156/1283 | 12.16%
Page: 157/1283 | 12.24%
Page: 158/1283 | 12.31%
Page: 159/1283 | 12.39%
Page: 160/1283 | 12.47%
Page: 161/1283 | 12.55%
Page: 162/1283 | 12.63%
Page: 163/1283 | 12.70%
Page: 164/1283 | 12.78%
Page: 165/1283 | 12.86%
Page: 166/1283 | 12.94%
Page: 167/1283 | 13.02%
Page: 168/1283 | 13.09%
Page: 169/1283 | 13.17%
Page: 170/1283 | 13.25%
Page: 171/1283 | 13.33%
Page: 172/1283 | 13.41%
Page: 173/1283 | 13.48%
Page: 174/1283 | 13.56%
Page: 175/1283 | 13.64%
Page: 176/1283 | 13.72%
Page: 177/1283 | 13.80%
Page: 178/1283 | 13.87%
Page: 179/1283 |

Page: 480/1283 | 37.41%
Page: 481/1283 | 37.49%
Page: 482/1283 | 37.57%
Page: 483/1283 | 37.65%
Page: 484/1283 | 37.72%
Page: 485/1283 | 37.80%
Page: 486/1283 | 37.88%
Page: 487/1283 | 37.96%
Page: 488/1283 | 38.04%
Page: 489/1283 | 38.11%
Page: 490/1283 | 38.19%
Page: 491/1283 | 38.27%
Page: 492/1283 | 38.35%
Page: 493/1283 | 38.43%
Page: 494/1283 | 38.50%
Page: 495/1283 | 38.58%
Page: 496/1283 | 38.66%
Page: 497/1283 | 38.74%
Page: 498/1283 | 38.82%
Page: 499/1283 | 38.89%
Page: 500/1283 | 38.97%
Page: 501/1283 | 39.05%
Page: 502/1283 | 39.13%
Page: 503/1283 | 39.20%
Page: 504/1283 | 39.28%
Page: 505/1283 | 39.36%
Page: 506/1283 | 39.44%
Page: 507/1283 | 39.52%
Page: 508/1283 | 39.59%
Page: 509/1283 | 39.67%
Page: 510/1283 | 39.75%
Page: 511/1283 | 39.83%
Page: 512/1283 | 39.91%
Page: 513/1283 | 39.98%
Page: 514/1283 | 40.06%
Page: 515/1283 | 40.14%
Page: 516/1283 | 40.22%
Page: 517/1283 | 40.30%
Page: 518/1283 | 40.37%
Page: 519/1283 | 40.45%
Page: 520/1283 | 40.53%
Page: 521/1283 |

Page: 822/1283 | 64.07%
Page: 823/1283 | 64.15%
Page: 824/1283 | 64.22%
Page: 825/1283 | 64.30%
Page: 826/1283 | 64.38%
Page: 827/1283 | 64.46%
Page: 828/1283 | 64.54%
Page: 829/1283 | 64.61%
Page: 830/1283 | 64.69%
Page: 831/1283 | 64.77%
Page: 832/1283 | 64.85%
Page: 833/1283 | 64.93%
Page: 834/1283 | 65.00%
Page: 835/1283 | 65.08%
Page: 836/1283 | 65.16%
Page: 837/1283 | 65.24%
Page: 838/1283 | 65.32%
Page: 839/1283 | 65.39%
Page: 840/1283 | 65.47%
Page: 841/1283 | 65.55%
Page: 842/1283 | 65.63%
Page: 843/1283 | 65.71%
Page: 844/1283 | 65.78%
Page: 845/1283 | 65.86%
Page: 846/1283 | 65.94%
Page: 847/1283 | 66.02%
Page: 848/1283 | 66.10%
Page: 849/1283 | 66.17%
Page: 850/1283 | 66.25%
Page: 851/1283 | 66.33%
Page: 852/1283 | 66.41%
Page: 853/1283 | 66.48%
Page: 854/1283 | 66.56%
Page: 855/1283 | 66.64%
Page: 856/1283 | 66.72%
Page: 857/1283 | 66.80%
Page: 858/1283 | 66.87%
Page: 859/1283 | 66.95%
Page: 860/1283 | 67.03%
Page: 861/1283 | 67.11%
Page: 862/1283 | 67.19%
Page: 863/1283 |

Page: 1157/1283 | 90.18%
Page: 1158/1283 | 90.26%
Page: 1159/1283 | 90.34%
Page: 1160/1283 | 90.41%
Page: 1161/1283 | 90.49%
Page: 1162/1283 | 90.57%
Page: 1163/1283 | 90.65%
Page: 1164/1283 | 90.72%
Page: 1165/1283 | 90.80%
Page: 1166/1283 | 90.88%
Page: 1167/1283 | 90.96%
Page: 1168/1283 | 91.04%
Page: 1169/1283 | 91.11%
Page: 1170/1283 | 91.19%
Page: 1171/1283 | 91.27%
Page: 1172/1283 | 91.35%
Page: 1173/1283 | 91.43%
Page: 1174/1283 | 91.50%
Page: 1175/1283 | 91.58%
Page: 1176/1283 | 91.66%
Page: 1177/1283 | 91.74%
Page: 1178/1283 | 91.82%
Page: 1179/1283 | 91.89%
Page: 1180/1283 | 91.97%
Page: 1181/1283 | 92.05%
Page: 1182/1283 | 92.13%
Page: 1183/1283 | 92.21%
Page: 1184/1283 | 92.28%
Page: 1185/1283 | 92.36%
Page: 1186/1283 | 92.44%
Page: 1187/1283 | 92.52%
Page: 1188/1283 | 92.60%
Page: 1189/1283 | 92.67%
Page: 1190/1283 | 92.75%
Page: 1191/1283 | 92.83%
Page: 1192/1283 | 92.91%
Page: 1193/1283 | 92.99%
Page: 1194/1283 | 93.06%
Page: 1195/1283 | 93.14%
Page: 1196/1283 | 93.22%


Page: 28/580 | 4.83%
Page: 29/580 | 5.00%
Page: 30/580 | 5.17%
Page: 31/580 | 5.34%
Page: 32/580 | 5.52%
Page: 33/580 | 5.69%
Page: 34/580 | 5.86%
Page: 35/580 | 6.03%
Page: 36/580 | 6.21%
Page: 37/580 | 6.38%
Page: 38/580 | 6.55%
Page: 39/580 | 6.72%
Page: 40/580 | 6.90%
Page: 41/580 | 7.07%
Page: 42/580 | 7.24%
Page: 43/580 | 7.41%
Page: 44/580 | 7.59%
Page: 45/580 | 7.76%
Page: 46/580 | 7.93%
Page: 47/580 | 8.10%
Page: 48/580 | 8.28%
Page: 49/580 | 8.45%
Page: 50/580 | 8.62%
Page: 51/580 | 8.79%
Page: 52/580 | 8.97%
Page: 53/580 | 9.14%
Page: 54/580 | 9.31%
Page: 55/580 | 9.48%
Page: 56/580 | 9.66%
Page: 57/580 | 9.83%
Page: 58/580 | 10.00%
Page: 59/580 | 10.17%
Page: 60/580 | 10.34%
Page: 61/580 | 10.52%
Page: 62/580 | 10.69%
Page: 63/580 | 10.86%
Page: 64/580 | 11.03%
Page: 65/580 | 11.21%
Page: 66/580 | 11.38%
Page: 67/580 | 11.55%
Page: 68/580 | 11.72%
Page: 69/580 | 11.90%
Page: 70/580 | 12.07%
Page: 71/580 | 12.24%
Page: 72/580 | 12.41%
Page: 73/580 | 12.59%
Page: 74/580 | 12.

Page: 389/580 | 67.07%
Page: 390/580 | 67.24%
Page: 391/580 | 67.41%
Page: 392/580 | 67.59%
Page: 393/580 | 67.76%
Page: 394/580 | 67.93%
Page: 395/580 | 68.10%
Page: 396/580 | 68.28%
Page: 397/580 | 68.45%
Page: 398/580 | 68.62%
Page: 399/580 | 68.79%
Page: 400/580 | 68.97%
Page: 401/580 | 69.14%
Page: 402/580 | 69.31%
Page: 403/580 | 69.48%
Page: 404/580 | 69.66%
Page: 405/580 | 69.83%
Page: 406/580 | 70.00%
Page: 407/580 | 70.17%
Page: 408/580 | 70.34%
Page: 409/580 | 70.52%
Page: 410/580 | 70.69%
Page: 411/580 | 70.86%
Page: 412/580 | 71.03%
Page: 413/580 | 71.21%
Page: 414/580 | 71.38%
Page: 415/580 | 71.55%
Page: 416/580 | 71.72%
Page: 417/580 | 71.90%
Page: 418/580 | 72.07%
Page: 419/580 | 72.24%
Page: 420/580 | 72.41%
Page: 421/580 | 72.59%
Page: 422/580 | 72.76%
Page: 423/580 | 72.93%
Page: 424/580 | 73.10%
Page: 425/580 | 73.28%
Page: 426/580 | 73.45%
Page: 427/580 | 73.62%
Page: 428/580 | 73.79%
Page: 429/580 | 73.97%
Page: 430/580 | 74.14%
Page: 431/580 | 74.31%
Page: 432/5

In [7]:
onerge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151389 entries, 0 to 151388
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Donemi_ve_Yasama_Yili    151389 non-null  object
 1   Esas_No                  151389 non-null  object
 2   Baskanliga_Gelis_Tarihi  151379 non-null  object
 3   Milletvekili             151389 non-null  object
 4   Ozeti                    151389 non-null  object
 5   Onerge_Grubu             151389 non-null  object
dtypes: object(6)
memory usage: 6.9+ MB


In [8]:
onerge_df.head()

,Donemi_ve_Yasama_Yili,Esas_No,Baskanliga_Gelis_Tarihi,Milletvekili,Ozeti,Onerge_Grubu
0,22/5,7/22994,01/06/2007,İSTANBUL MİLLETVEKİLİ EMİN ŞİRİN,Telsim'in borçlularına yönelik bir uygulamasın...,yazılı
1,22/5,7/22987,01/06/2007,DENİZLİ MİLLETVEKİLİ MUSTAFA GAZALCI,Denizli'deki kuraklık mağduriyetine ilişkin.,yazılı
2,22/5,7/22986,01/06/2007,DİYARBAKIR MİLLETVEKİLİ MUHSİN KOÇYİĞİT,Diyarbakır-Bismil'deki pamuk üreticilerinin de...,yazılı
3,22/5,7/22985,01/06/2007,DENİZLİ MİLLETVEKİLİ MUSTAFA GAZALCI,Basında çıkan Bağcılar Lisesiyle ilgili iddial...,yazılı
4,22/5,7/22984,01/06/2007,İSTANBUL MİLLETVEKİLİ MEHMET ALİ ÖZPOLAT,Basında çıkan Bağcılar Lisesiyle ilgili iddial...,yazılı


In [9]:
onerge_df.tail()

,Donemi_ve_Yasama_Yili,Esas_No,Baskanliga_Gelis_Tarihi,Milletvekili,Ozeti,Onerge_Grubu
151384,26/1,6/5,25/11/2015,GAZİANTEP MİLLETVEKİLİ CANAN CANDEMİR ÇELİK,Koruyucu aile sistemi ile ilgili verilere ve k...,sözlü
151385,26/1,6/4,25/11/2015,GAZİANTEP MİLLETVEKİLİ CANAN CANDEMİR ÇELİK,Çocuk koruma hizmetlerine ilişkin,sözlü
151386,26/1,6/3,25/11/2015,İSTANBUL MİLLETVEKİLİ TÜLAY KAYNARCA,Çocuklara ve engellilere yönelik hizmetlere il...,sözlü
151387,26/1,6/2,25/11/2015,İSTANBUL MİLLETVEKİLİ TÜLAY KAYNARCA,Sosyal yardım programları ile suça sürüklenen ...,sözlü
151388,26/1,6/1,25/11/2015,GAZİANTEP MİLLETVEKİLİ CANAN CANDEMİR ÇELİK,Dezavantajlı gruplara yönelik çalışmalar ile y...,sözlü


In [10]:
onerge_df.to_csv("onergeler_yazili_sozlu_22_26.csv", index=False, encoding="utf-8")

In [11]:
driver.quit()